### 커피 특징
* !원두 이름!
    - 타입 (string)
    - 로스팅 포인트 (string)
    - 지속가능성 (string)
    - 컵 노트 (list of strings)
    - 바디감
    - 신맛
    - 단맛
    - 쓴맛

In [1]:
# package import
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import re
import time
# from tqdm import tqdm_notebook
from tqdm import tqdm

import pickle
import itertools

In [46]:
# # example dictionary into df
# dict = {'원두1':{'타입':'블랜드', '로스팅 포인트':'다크', '지속가능성':'-', '컵노트':['구운 아몬드', '밀크 초콜릿'], '바디감':4, '신맛':2, '단맛':4, '쓴맛':3}}
# # df = pd.DataFrame.from_dict(dict)
# # df.T

# # dict 추가
# dict['원두1']['테스트'] = '테스트'
### dict {'name':'원두1', ...}

In [112]:
service = Service(path= './driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [113]:
# 메인 페이지
url = 'https://biz.koke.kr/products?filters=%7B%22categoryGroup%22%3A%22WholeBean%22%7D'
driver.get(url) # go to website
time.sleep(1)

In [114]:
links = [] # 제품 링크 리스트

# 약 4분 30초 소요
for i in range(31):
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col.w-full.lg\:w-3\/4")
    link_elements = raw[0].find_elements(By.TAG_NAME,'a') 
    href_links = [link.get_attribute('href') for link in link_elements]
    links = links + href_links # 현재 페이지에 있는 상품 가져오기
    print('페이지수', i, '링크 수', len(href_links), '총 링크 수', len(links))
    time.sleep(3)

    if i is not 30:
        button = driver.find_elements(By.CSS_SELECTOR ,"button.text-gray-500.bg-white.border.border-gray-200.koke-button")
        print(len(button))
        button[-1].click()
        time.sleep(5)

links = set(links)
print(len(links)) # 총 556개

driver.quit()

<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\Playdata\AppData\Local\Temp\ipykernel_1956\2079587560.py:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if i is not 30:


페이지수 0 링크 수 18 총 링크 수 18
2
페이지수 1 링크 수 18 총 링크 수 36
2
페이지수 2 링크 수 18 총 링크 수 54
3
페이지수 3 링크 수 18 총 링크 수 72
3
페이지수 4 링크 수 18 총 링크 수 90
3
페이지수 5 링크 수 18 총 링크 수 108
3
페이지수 6 링크 수 18 총 링크 수 126
3
페이지수 7 링크 수 18 총 링크 수 144
3
페이지수 8 링크 수 18 총 링크 수 162
3
페이지수 9 링크 수 18 총 링크 수 180
3
페이지수 10 링크 수 18 총 링크 수 198
3
페이지수 11 링크 수 18 총 링크 수 216
3
페이지수 12 링크 수 18 총 링크 수 234
3
페이지수 13 링크 수 18 총 링크 수 252
3
페이지수 14 링크 수 18 총 링크 수 270
3
페이지수 15 링크 수 18 총 링크 수 288
3
페이지수 16 링크 수 18 총 링크 수 306
3
페이지수 17 링크 수 18 총 링크 수 324
3
페이지수 18 링크 수 18 총 링크 수 342
3
페이지수 19 링크 수 18 총 링크 수 360
3
페이지수 20 링크 수 18 총 링크 수 378
3
페이지수 21 링크 수 18 총 링크 수 396
3
페이지수 22 링크 수 18 총 링크 수 414
3
페이지수 23 링크 수 18 총 링크 수 432
3
페이지수 24 링크 수 18 총 링크 수 450
3
페이지수 25 링크 수 18 총 링크 수 468
3
페이지수 26 링크 수 18 총 링크 수 486
3
페이지수 27 링크 수 18 총 링크 수 504
3
페이지수 28 링크 수 18 총 링크 수 522
3
페이지수 29 링크 수 18 총 링크 수 540
3
페이지수 30 링크 수 16 총 링크 수 556
556


In [4]:
# # Pickle dump the set of links to a file
# with open("links.pkl", "wb") as file:
#     pickle.dump(links, file)

NameError: name 'links' is not defined

In [5]:
# Load the set of links from the file
with open("links.pkl", "rb") as file:
    loaded_links = pickle.load(file)

# Print the loaded set of links
# print("Loaded links:", loaded_links)

In [6]:
len(loaded_links)

556

***

### 제품 페이지에서의 데이터 수집

In [2]:
# class
def level_data(html):
    # Parse the HTML content
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div elements without the 'bg-opacity-30' class
    divs_without_opacity_30 = [div for div in soup.find_all('div', class_='bg-koke-green') if 'bg-opacity-30' not in div.get('class')]

    # Total number of blocks
    total_blocks = 5

    # Initialize position (n) to None
    position = None

    # Check if there are any elements without the 'bg-opacity-30' class
    if divs_without_opacity_30:
        # Get the index of the first element (0-based) and add 1 to get the position (1-based)
        position = divs_without_opacity_30[0].find_parent().find_all('div', class_='bg-koke-green').index(divs_without_opacity_30[0]) + 1

    #print(f"The element without 'bg-opacity-30' class is at position {position}/{total_blocks}")
    return position

In [3]:
def other(driver):
    type = ['블렌드', '싱글오리진', '디카페인']
    roast = ['라이트', '라이트미디엄', '미디엄', '미디엄다크', '다크']
    sustain = ['직접무역', '공정무역', '유기농']
    tmp = {}
    
    elements = driver.find_elements(By.CSS_SELECTOR, 'li.lg\:block')
    
    result = []
    for i in range(len(elements)):
        html = elements[i].get_attribute('innerHTML')
        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')
    
        if soup.find_all('svg', 'hidden'):
            result.append(elements[i].text)
    
    for i in result:
        if i in type:
            tmp['type'] = i
        elif i in roast:
            tmp['roast'] = i
        else: # i in sustain:
            tmp['sustain'] = i
    return tmp
            

In [35]:
# loaded_links
# small_set = set(itertools.islice(loaded_links, 3))

In [82]:
len(loaded_links)

556

In [7]:
links_list = list(loaded_links)

In [8]:
len(links_list)

556

In [15]:
service = Service(path= './driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [10]:
data = {}
for link in tqdm(links_list):
    name = {}
    driver.get(link)
    time.sleep(3)
    
    # name
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col")
    coffee_name = raw[0].find_element(By.TAG_NAME,'h1').text
    data[coffee_name] =  dict.fromkeys(['타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛'])  # add coffee names as keys and empty dictionary as values
    time.sleep(3)
    
    # add data
    temp = driver.find_elements(By.CSS_SELECTOR,'div.flex.justify-between.w-full.h-2\.5.overflow-hidden.rounded-full') # len(temp) = 4
    keys = ['바디감', '신맛', '단맛', '쓴맛']
    for i in range(4):
        html = temp[i].get_attribute('innerHTML')
        data[coffee_name][keys[i]] = level_data(html)
    time.sleep(3)
    
    # others
    tmp = other(driver)
    for i in range(len(tmp)):
        keys = list(tmp.keys())
        if keys[i] == 'type':
            data[coffee_name]['타입'] = tmp['type']
        elif keys[i] == 'roast':
            data[coffee_name]['로스팅 포인트'] = tmp['roast']
        else:
            data[coffee_name]['지속가능성'] = tmp['sustain']
    time.sleep(3)
            
    # 컵 노트
    try:
        notes = driver.find_element(By.CSS_SELECTOR, 'li.tracking-wide.koke-word-keep-all')
        data[coffee_name]['컵 노트'] = notes
    except:
      data[coffee_name]['컵 노트'] = '-'
      
    time.sleep(1)       
    
driver.quit()

  0%|          | 0/556 [00:00<?, ?it/s]

100%|██████████| 556/556 [2:08:42<00:00, 13.89s/it]  


In [11]:
len(data.keys())

537

In [12]:
list(data.keys())[0]

'과테말라 디카페인'

In [23]:
type(data)

dict

In [27]:
df = pd.DataFrame.from_dict(data)
newdf = df.T

In [28]:
newdf.to_csv('data.csv')

In [16]:
names = {}
for link in tqdm(links_list):
    name = {}
    driver.get(link)
    time.sleep(3)
    
    # name
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col")
    coffee_name = raw[0].find_element(By.TAG_NAME,'h1').text
    names[coffee_name] =  dict.fromkeys(['타입', '로스팅 포인트', '지속가능성', '컵 노트', '바디감', '신맛', '단맛', '쓴맛'])  # add coffee names as keys and empty dictionary as values
    time.sleep(0.5)

100%|██████████| 556/556 [37:56<00:00,  4.09s/it]


In [19]:
name_ls = list(names.keys())

In [20]:
len(name_ls)

537